In [1]:
import sys
import os
from functools import partial

import getpass
import requests
import json
import time
from pprint import pprint

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import (ConversationBufferMemory,
                              ConversationBufferWindowMemory,
                              ConversationSummaryMemory,
                              VectorStoreRetrieverMemory)
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

from dotenv import load_dotenv
# Load environment variables
load_dotenv('../.env')

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [2]:
class SimpleChatBot:
    key_type_memory = {
        'summary': ConversationSummaryMemory,
        'simple': ConversationBufferMemory,
        'window': partial(ConversationBufferWindowMemory, k=3),
    }

    def __init__(self, llm, memory_type):
        self.llm = llm
        self.memory = self.key_type_memory[memory_type](llm=llm)
        self.conversation = ConversationChain(
            llm=self.llm,
            verbose=False,
            memory=self.memory,
        )

    def _respond(self, user_input):
        response = self.conversation.predict(input=user_input)
        return response
        

    def print_memory(self):
        print(self.memory.load_memory_variables({}))

    def run(self):
        print("This is a simple chat bot:")
        while True:
            line = input('User: ')
            if line == '':
                break
            print(f'User: {line}')
            response = self._respond(line)
            print(f'AI: {response}')


In [ ]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(llm=giga, memory_type='summary')
chat.run()
pprint(chat.print_memory())

Now let's check how it works with each type of memory

In [3]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window')
chat.run()
pprint(chat.print_memory())

/tmp/ipykernel_20219/2173066221.py:11: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  self.conversation = ConversationChain(


This is a simple chat bot:
User: Tell me something about Bashkortostan.
AI: Башкортостан — это республика в составе Российской Федерации, расположенная на Южном Урале и в части Предуралья. Столица республики — город Уфа. Регион известен своей богатой историей, культурой и природой. Здесь проживают представители более 100 национальностей, каждая из которых внесла свой вклад в развитие региона. Башкортостан славится производством нефти и газа, а также развитым сельским хозяйством. В Уфе ежегодно проходят международные форумы и конференции, что делает город важным центром для международного сотрудничества.
User: What is the capital of that region?
AI: The capital of Bashkortostan is Ufa.
User: What do you know about the capital?
AI: Извините, но у меня нет информации о конкретной столице республики Башкортостан, Уфе. Можете ли вы уточнить ваш запрос?
User: What is most wide spread ethnicity in this region?
AI: Извините, но у меня нет информации о конкретном этническом большинстве в регион

In [6]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'simple')
chat.run()
pprint(chat.print_memory())

This is a simple chat bot:
User: Tell me something about Bashkortostan.
AI: Башкортостан — это республика в составе Российской Федерации, расположенная на Южном Урале и в части Предуралья. Столица республики — город Уфа. Башкортостан известен своей богатой историей, культурой и природой. Здесь проживают представители различных национальностей, которые бережно хранят свои традиции и обычаи. Экономика региона базируется на нефтедобыче, машиностроении, сельском хозяйстве и туризме. В Башкортостане также находится множество природных заповедников и национальных парков, таких как Башкирский государственный заповедник и национальный парк "Башкирия".
User: What is the capital of that region?
AI: Столицей региона Башкортостан является город Уфа.
User: What do you know about Ufa?
AI: Извините, но у меня нет информации о городе Уфа в данный момент. Могу ли я помочь вам с чем-то еще?
User: What is most wade spread ethnicity in this region?
AI: Извините, но у меня нет информации о наиболее распрос

# Отчет о различиях типов памяти

## 1. ConversationBufferMemory
- **Описание**: Хранит полную историю диалога в виде буфера. Каждый новый ввод добавляется в конец этого буфера.
- **Применение**: Используется, когда важно сохранять все сообщения для контекста, чтобы не терялись никакие детали в течение диалога.
- **Преимущества**: Сохраняет всю историю, что может быть полезно для отслеживания всех аспектов беседы.
- **Недостатки**: По мере увеличения объема данных, буфер может стать слишком большим и замедлить обработку.

## 2. ConversationBufferWindowMemory
- **Описание**: Хранит только последние *n* сообщений из истории диалога (окно). Вместо хранения всей истории сохраняются только недавние сообщения.
- **Применение**: Когда важен недавний контекст, а не полная история диалога, например, для кратких ответов или выполнения простых задач.
- **Преимущества**: Экономит память и ускоряет обработку за счет хранения только нужного количества сообщений.
- **Недостатки**: Старые сообщения теряются, что может быть проблемой, если важно помнить весь контекст.

## 3. ConversationSummaryMemory
- **Описание**: Создает сжатое резюме диалога, извлекая ключевые моменты из истории беседы. Вместо сохранения всей истории или окна сообщений, генерируется краткая сводка.
- **Применение**: Полезно, когда нужен краткий обзор важнейших моментов длительного диалога, без необходимости хранения каждого сообщения.
- **Преимущества**: Экономит память за счет хранения резюме вместо полного текста диалога.
- **Недостатки**: Возможны потери некоторых деталей, которые могут оказаться важными для дальнейшего контекста беседы.

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [4]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
import requests
from pprint import pprint
import os

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

In [6]:
city = 'Moscow'
city = 'Санкт-Петербург'
city = 'Saint Petersburg'
url = (
        f"https://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={openweather_key}&units=metric"
    )
response = requests.get(url)
pprint(response.json())

{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 59.8944, 'lon': 30.2642},
 'dt': 1729613941,
 'id': 498817,
 'main': {'feels_like': 11.01,
          'grnd_level': 1012,
          'humidity': 80,
          'pressure': 1015,
          'sea_level': 1015,
          'temp': 11.7,
          'temp_max': 11.7,
          'temp_min': 11.08},
 'name': 'Saint Petersburg',
 'sys': {'country': 'RU',
         'id': 197864,
         'sunrise': 1729572944,
         'sunset': 1729607442,
         'type': 2},
 'timezone': 10800,
 'visibility': 10000,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 260, 'speed': 5}}


In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WeatherResult(BaseModel):
    message: str = Field(description="Message about the weather status")
    temperature: float = Field(description="Real temperature")
    feels_like: float = Field(description="How the weather temperature feels like")
    description: str = Field(description="General weather description")


few_shot_examples = [
    {
        "request": "What is the weather in Moscow",
        "params": {"city": "Moscow"},
    },
    {
        "request": "Give me the weather in Samara",
        "params": {"city": "Samara"},
    },
    {
        "request": "Tell the temperature in Saint Petersburg",
        "params": {"city": "Saint Petersburg"},
    },
]

@tool(few_shot_examples=few_shot_examples)
def get_weather(city: str) -> str:
    """Return weather information for the given city"""
    url = (
        f"https://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={openweather_key}&units=metric"
    )
    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        weather_description = data['weather'][0]['description']
        temp = data['main']['temp']
        feels_like = data['main']['feels_like']
        
        return WeatherResult(
            message="Погода была успешно получена.",
            temperature=temp,
            feels_like=feels_like,
            description=f'{weather_description}'
        )
    else:
        return WeatherResult(
            message="Не удалось получить данные о погоде. Проверьте название города или API-ключ.",
            temperature=0,
            feels_like=0,
            description="No description"
        )

city_name = "Москва"
city_name = "Moscow"
print(get_weather(city_name))


/tmp/ipykernel_57983/574559311.py:65: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  print(get_weather(city_name))


{'base': 'stations',
 'clouds': {'all': 100},
 'cod': 200,
 'coord': {'lat': 55.7522, 'lon': 37.6156},
 'dt': 1729613573,
 'id': 524901,
 'main': {'feels_like': 6.53,
          'grnd_level': 1002,
          'humidity': 61,
          'pressure': 1021,
          'sea_level': 1021,
          'temp': 8.82,
          'temp_max': 9.1,
          'temp_min': 8.1},
 'name': 'Moscow',
 'sys': {'country': 'RU',
         'id': 2094500,
         'sunrise': 1729570498,
         'sunset': 1729606361,
         'type': 2},
 'timezone': 10800,
 'visibility': 10000,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 262, 'gust': 10.32, 'speed': 4.04}}
message='Погода была успешно получена.' temperature=8.82 feels_like=6.53 description='overcast clouds'


In [8]:

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function
        self.tools = [self.agent_function]
        self.agent = create_gigachat_functions_agent(self.llm, self.tools)
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=False,
        )

    def run(self, user_input: str):
        result = self.agent_executor.invoke({
            "input": user_input
            })
        return result['output']

In [9]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_weather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)

{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 45.0428, 'lon': 41.9733},
 'dt': 1729613861,
 'id': 487846,
 'main': {'feels_like': 4.72,
          'grnd_level': 975,
          'humidity': 93,
          'pressure': 1030,
          'sea_level': 1030,
          'temp': 6.16,
          'temp_max': 6.16,
          'temp_min': 6.16},
 'name': 'Stavropol',
 'sys': {'country': 'RU',
         'id': 8968,
         'sunrise': 1729568226,
         'sunset': 1729606541,
         'type': 1},
 'timezone': 10800,
 'visibility': 10000,
 'weather': [{'description': 'broken clouds',
              'icon': '04n',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 320, 'speed': 2}}


'Сейчас в Ставрополе облачно, температура около 6 градусов Цельсия, ощущается как около 5 градусов.'

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [13]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
import json
import datetime
from dateutil import parser

openweather_key = os.environ.get("OPENWEATHER_API_KEY")


In [14]:
def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)

In [15]:
class HotelBookingResult(BaseModel):
    url: str = Field(description="URL for booking hotel")
    city: str = Field(description="City for booking hotel")
    date_in: str = Field(description="Start date")
    date_out: str = Field(description="End date")
    message: str = Field(description="message about hotel booking URL generation result")

few_shot_examples_hotel = [
    {
        "request": "What is a URL for booking a hotel in Moscow from 10.12.23 to 15.12.23",
        "params": {
            "city": "Moscow",
            "date_in_str": "10.12.23",
            "date_out_str": "15.12.23"
            },
    },
    {
        "request": "Give me a link for booking hotel in Saint Petersburg from 15.06.24 to 15.07.24",
        "params": {
            "city": "Saint Petersburg",
            "date_in_str": "15.06.24",
            "date_out_str": "15.07.24"
            },
    },
    {
        "request": "Can you give me a web page for hotel reserving in Samara from 01.01.22 to 21.01.22",
        "params": {
            "city": "Samara",
            "date_in_str": "01.01.22",
            "date_out_str": "21.01.22"
            },
    },
]

@tool(few_shot_examples=few_shot_examples_hotel)
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> str:
    "Return hotel booking url for the given city, start date and end date"
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?'
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    return HotelBookingResult(
        message='URL succesfully generated',
        url=url,
        city=city,
        date_in=date_in_str,
        date_out=date_out_str,
    )
    # return f'Here is your URL for booking: {url} in {city} on {date_in_str} / {date_out_str}'


In [16]:
class TicketsBookingResult(BaseModel):
    url: str = Field(description="URL for booking tickets")
    city_from: str = Field(description="Departing city")
    date_in: str = Field(description="Start date")
    date_out: str = Field(description="End date")
    message: str = Field(description="message about hotel booking URL generation result")


few_shot_examples_tickets = [
    {
        "request": "Can you provide a link for booking tickets from Moscow to Saint Petersburg on 20.11.23 with return on 25.11.23?",
        "params": {
            "city_from": "Moscow",
            "city_to": "Saint Petersburg",
            "date_in_str": "20.11.23",
            "date_out_str": "25.11.23"
        },
    },
    {
        "request": "Give me a URL to buy tickets from Kazan to Sochi departing on 01.03.24 and returning on 10.03.24",
        "params": {
            "city_from": "Kazan",
            "city_to": "Sochi",
            "date_in_str": "01.03.24",
            "date_out_str": "10.03.24"
        },
    },
    {
        "request": "I need a link to book a flight from Novosibirsk to Vladivostok on 05.05.24 and return on 15.05.24",
        "params": {
            "city_from": "Novosibirsk",
            "city_to": "Vladivostok",
            "date_in_str": "05.05.24",
            "date_out_str": "15.05.24"
        },
    },
    {
        "request": "Can you give me a ticket booking link from Yekaterinburg to Omsk for 01.12.23 with return on 05.12.23?",
        "params": {
            "city_from": "Yekaterinburg",
            "city_to": "Omsk",
            "date_in_str": "01.12.23",
            "date_out_str": "05.12.23"
        },
    },
]


@tool(few_shot_examples=few_shot_examples_tickets)
def get_url_booking_tikets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> str:
    "Return tikets booking url for the given start city, end city, start date and end date"
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?'
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    return TicketsBookingResult(
        message='URL succesfully generated',
        url=url,
        city_from=city_from,
        city_to=city_to,
        date_in=date_in_str,
        date_out=date_out_str,
    )
    # return f'Here is your url for tickets ordering: {url} from {city_from} to {city_to} on {date_in_str} / {date_out_str}'


In [17]:
class MultiAgent:
    def __init__(self, llm, agent_function_wheater, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_wheater = agent_function_wheater
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets
        self.weather_agent = self._create_agent(
            tools=[self.agent_function_wheater],
        )
        self.hotel_agent = self._create_agent(
            tools=[self.agent_function_hotels],
        )
        self.tickets_agent = self._create_agent(
            tools=[self.agent_function_tickets],
        )

    def _create_agent(self, tools):
        agent = create_gigachat_functions_agent(self.llm, tools)
        return AgentExecutor(agent=agent, tools=tools, verbose=True,)

    def run(self, user_input: str):
        
        weather = self.weather_agent.invoke({"input": user_input})
        tickets = self.tickets_agent.invoke({"input": user_input})
        hotel = self.hotel_agent.invoke({"input": user_input})
        answer = f"{weather['output']}\n{tickets['output']}\n{hotel['output']}\n"
        return answer


In [18]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_weather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "I want to go to the trip from Moscow to Saint Petersburg from 10.11.24 to 28.11.24. Can you give me url for booking hotel and tickets. And tell me about the weather in Saint Petersburg please."
answer = traveler.run(user_input)
print(answer)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Saint Petersburg'}`


{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 59.8944, 'lon': 30.2642},
 'dt': 1729615681,
 'id': 498817,
 'main': {'feels_like': 10.4,
          'grnd_level': 1012,
          'humidity': 80,
          'pressure': 1015,
          'sea_level': 1015,
          'temp': 11.14,
          'temp_max': 11.14,
          'temp_min': 11.08},
 'name': 'Saint Petersburg',
 'sys': {'country': 'RU',
         'id': 197864,
         'sunrise': 1729572944,
         'sunset': 1729607442,
         'type': 2},
 'timezone': 10800,
 'visibility': 10000,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 260, 'speed': 5}}
message='Погода была успешно получена.' temperature=11.14 feels_like=10.4 description='clear sky'Извините за путаницу, я обработал только часть вашего запроса и не смо